In [1]:
from sentence_transformers import SentenceTransformer, util
import os
os.chdir('../')
from general_dataset import GeneralDataset
from torch.utils.data import DataLoader
import torch
from agi_utils import *
import openai
import numpy as np
from IPython.utils import io
import random
from tqdm import tqdm
from evaluate import load
from torchvision import transforms
from transformers import AutoModel, AutoFeatureExtractor
from torchmetrics.multimodal import CLIPScore
from combine_module_seq import SeqCombine

/common/home/yg334/anaconda3/envs/agi/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [2]:
"""
assign openagi data path 
"""
# data_path = "YOUR_DATA_PATH"
data_path = "/common/users/yg334/openagi_data/"


task_discriptions = txt_loader("./task_description.txt")
# task_idx = [0,21,61,105,110,120,10,35,62,107,115]
test_task_idx = [2,3,10,15,20,35,45,55,65,70,70,90,106,107]
test_dataloaders = []
for i in test_task_idx:
    dataset = GeneralDataset(i, data_path)
    dataloader = DataLoader(dataset, batch_size=5)
    test_dataloaders.append(dataloader)
    
test_tasks = [task_discriptions[i].strip() for i in test_task_idx]

In [3]:
train_solution = []
with open('./train_model_sequence.txt') as f:
    lines = f.readlines()
    for line in lines[:50]:
        train_solution.append(line)
f.close()

train_tasks = []
with open('./train_task_description.txt') as f:
    lines = f.readlines()
    for line in lines[:50]:
        train_tasks.append(line)
f.close()

context = ""
for i in range(len(train_tasks)):
    steps = ""
    for index,j in enumerate(train_solution[i].split(',')):
        steps += "Step " + str(index+1) + ":" + j.strip("\n") + ", \n"
    cur = "Problem: " + train_tasks[i] +  "Solution:\n" + steps
    context += cur
    
# print(context + "Problem: " + test_tasks[0]+"\nSoltuion: ")

In [4]:
# device_list = ["cuda:1","cuda:2","cuda:3","cuda:4","cuda:5","cuda:7","cpu"]
device_list = ["cuda:5","cuda:6","cpu"]
seqCombination = SeqCombine(device_list)


# Load a pre-trained Vision Transformer model and its feature extractor
vit_ckpt = "nateraw/vit-base-beans"
vit = AutoModel.from_pretrained(vit_ckpt)
vit.eval()
vit_extractor = AutoFeatureExtractor.from_pretrained(vit_ckpt)

f = transforms.ToPILImage()
bertscore = load("bertscore")
clip_score = CLIPScore(model_name_or_path="openai/clip-vit-base-patch16")

Fetching 20 files:   0%|          | 0/20 [00:00<?, ?it/s]

The config attributes {'scaling_factor': 0.18215} were passed to AutoencoderKL, but are not expected and will be ignored. Please verify your config.json configuration file.
Some weights of the model checkpoint at nateraw/vit-base-beans were not used when initializing ViTModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ViTModel were not initialized from the model checkpoint at nateraw/vit-base-beans and are newly initialized: ['vit.pooler.dense.bias', 'vit.pooler.dense.weight']
You should probably TRAIN this model on a down-strea

In [5]:
"""
assign openai api_key
"""
openai.api_key = "YOUR_API_KEY"


device = "cuda:7"
rewards = []
clips = []
berts = []
similairies = []

sentence_model = SentenceTransformer('all-MiniLM-L6-v2', device="cpu")

problem = test_tasks[1]


for i, task_description in enumerate(tqdm(test_tasks)):
    
    task_rewards = []
    with torch.no_grad():
        completion = openai.ChatCompletion.create(model="gpt-4",\
                                                  messages=[{"role": "user",\
                                                             "content": context +\
                                                             "Problem: " +\
                                                             task_description +\
                                                             "\nSoltuion: "}]
                                                  )
        gpt_output = completion.choices[0].message['content'].split('\n')
        gpt_steps = []
        for l,j in enumerate(gpt_output):
            if j[0:4] == "Step":
                gpt_steps.append(gpt_output[l])
        module_list = match_module_seq(gpt_steps, sentence_model)
    print(module_list)
    # break


    if len(module_list) > 0 and whole_module_seq_filter(module_list, test_task_idx[i]):
        seqCombination.construct_module_seq(module_list)

        for idx, batch in enumerate(test_dataloaders[i]):
            inputs = list(batch['input'][0])
            predictions = seqCombination.run_module_seq(inputs)

            if 0<=test_task_idx[i]<=14:
                outputs = list(batch['output'][0])
                dist = image_similarity(predictions, outputs, vit, vit_extractor)
                task_rewards.append(dist/100)
            elif 15<=test_task_idx[i]<=104 or 107<=test_task_idx[i]<=154:
                outputs = list(batch['output'][0])
                f1 = np.mean(txt_eval(predictions, outputs, bertscore, device=device))
                
                task_rewards.append(f1)
            else:
                score = score = clip_score(predictions, inputs)
                task_rewards.append(score.detach()/100)
                
        ave_task_reward = np.mean(task_rewards)    
        
        
        seqCombination.close_module_seq()
            
    else:
        ave_task_reward = 0
    print(ave_task_reward)
        
    if 0<=test_task_idx[i]<=14:
        similairies.append(ave_task_reward)
    elif 15<=test_task_idx[i]<=104 or 107<=test_task_idx[i]<=154:
        berts.append(ave_task_reward)
    else:
        clips.append(ave_task_reward)

    rewards.append(ave_task_reward)     
    

print("Finished testing!")    

  0%|                                                    | 0/14 [00:00<?, ?it/s]

Image Super Resolution, Image Deblurring, Colorization


  7%|███                                        | 1/14 [01:53<24:29, 113.07s/it]

0.6998428001403809
Image Deblurring, Colorization


 14%|██████▎                                     | 2/14 [02:16<12:07, 60.60s/it]

...

  0%|          | 0/50 [00:00<?, ?it/s]

 93%|███████████████████████████████████████   | 13/14 [28:39<04:36, 276.03s/it]

0.30556625
Fill Mask, Text Summarization, Machine Translation


100%|██████████████████████████████████████████| 14/14 [29:33<00:00, 126.70s/it]

0.25208689510822296
Finished testing!


In [7]:
np.mean(clips), np.mean(berts), np.mean(similairies), (np.mean(clips) + np.mean(berts) + np.mean(similairies))/3

(0.30556625, 0.6307922720387579, 0.6480373134613037, 0.5281319455926617)